In [1]:
import pandas as pd

data = pd.read_csv("../one_hot_encoded_data.csv", index_col=[0])

In [2]:
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,...,debt_settlement_flag_N,debt_settlement_flag_Y,settlement_status_ACTIVE,settlement_status_BROKEN,settlement_status_COMPLETE,settlement_status_No,next_pymnt_d_Apr-2019,next_pymnt_d_Feb-2019,next_pymnt_d_Mar-2019,next_pymnt_d_Unkown
0,10000.0,10000.0,10000.0,4.0,307.27,10.0,55000.0,Current,19.22,0.0,...,1,0,0,0,0,1,0,0,1,0
1,35000.0,35000.0,35000.0,18.0,770.63,10.0,200000.0,Current,17.80,1.0,...,1,0,0,0,0,1,0,0,1,0
2,9600.0,9600.0,9600.0,8.0,300.13,3.0,90000.0,Current,25.59,1.0,...,1,0,0,0,0,1,0,0,1,0
3,18000.0,18000.0,18000.0,6.0,557.52,10.0,70000.0,Current,5.55,0.0,...,1,0,0,0,0,1,0,0,1,0
4,12000.0,12000.0,12000.0,10.0,378.59,3.0,36500.0,Current,33.61,0.0,...,1,0,0,0,0,1,0,0,1,0


In [3]:
X = data.drop(columns=["loan_status"])

data["loan_status"] = data["loan_status"].astype("category")
category = data["loan_status"].cat.categories

y = data["loan_status"].cat.codes

In [4]:
y.unique()

array([1, 3, 6, 5, 4, 0, 2])

In [5]:
category
category[y]

Index(['Current', 'Current', 'Current', 'Current', 'Current', 'Current',
       'Current', 'Current', 'Current', 'Current',
       ...
       'Current', 'Late (31-120 days)', 'Current', 'Current', 'Current',
       'Current', 'Current', 'Current', 'Current', 'Current'],
      dtype='object', length=128194)

#### Xgboost manages only numeric vectors.
https://xgboost.readthedocs.io/en/latest/R-package/discoverYourData.html


In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)



# Xgboost manages only numeric vectors.
dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
dtest = xgb.DMatrix(X_test.values, label=y_test.values)

params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 7,
    'gamma': 0.1, 
    'max_depth': 6,
    'lambda': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'silent': 1,
    'eta': 0.1,
    'seed': 1000,
    'nthread': 4,
}

plst = params.items()
num_round = 10

bst = xgb.train(plst, dtrain, num_round)



y_pred = bst.predict(dtest)

score = accuracy_score(y_test, y_pred)

print("The accuracy score is: ", score)

The accuracy score is:  0.98927415265806


Cross Validation

In [8]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


# CV model
model = xgb.XGBClassifier()
kfold = KFold(n_splits=4, random_state=666)
results = cross_val_score(model, X, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 98.68% (0.26%)


In [26]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_scaled = ss.fit_transform(X)
# CV model
model = xgb.XGBClassifier()
kfold = KFold(n_splits=4, random_state=666)
results = cross_val_score(model, X_scaled, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Accuracy: 98.68% (0.26%)


In [25]:


X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

# print(type(X_train_scaled))
# Xgboost manages only numeric vectors.
dtrain_scaled = xgb.DMatrix(X_train_scaled, label=y_train.values)
dtest_scaled = xgb.DMatrix(X_test_scaled, label=y_test.values)

bst2 = xgb.train(plst, dtrain_scaled, num_round)
y_pred2 = bst2.predict(dtest_scaled)



# print(type(y_pred2))

score2 = accuracy_score(y_test, y_pred2)

print("The accuracy score is: ", score2)

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


The accuracy score is:  0.98927415265806


In [27]:
print(accuracy_score(y_pred, y_pred2))

1.0


In [16]:
print(y_pred2.shape)
print(y_test.shape)
print(y_pred.shape)
print(X_train_scaled.shape)

(102555,)
(25639,)
(25639,)
(102555, 267)
